In [2]:
import pandas as pd 
import pickle 
from libray import *
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
print (f"Pandas version {pd.__version__}")

Pandas version 1.5.2


In [4]:
print (f"Tensorflow version: {tf.__version__}")

Tensorflow version: 2.9.1


In [5]:
def load(file_name):
    b = {}
    try:
        b = pickle.load(open(file_name, "rb"))
        print("Loading Successful")
        return b
    except (OSError, IOError) as e:
        print("Loading Failed. Initializing to empty")
        b = {}
        return b

In [6]:
RFC = load("../models/RFC_haikal_for_Ws.pickle")
NCC = load("../models/PCR_haikal_for_Ws.pickle")


Loading Successful
Loading Successful


In [7]:
DNN = tf.keras.models.load_model("../models/TF2_model_for_Ws.h5")

: 

: 

In [ ]:
NCC.coefs_[0].mean(axis=1)

array([-24.46021681,   1.17478964,   0.08034624,   0.47741979,
        -1.70503827,   0.93513041, -10.86875436,   2.23682232])

In [ ]:
RFC.feature_importances_

array([0.14005727, 0.17810914, 0.0734944 , 0.16473502, 0.19207643,
       0.17126157, 0.0366392 , 0.04362697])

In [ ]:
DNN.weights[0].numpy().mean(axis=1)

array([-8.3203621e+00, -2.3365001e-01,  1.2053961e-01, -1.0901808e-03,
        1.4162257e-01, -2.9330578e-01, -2.1581888e-01, -1.2443406e+01],
      dtype=float32)

In [ ]:
#['CONSRANK_val','CP_HLPL','CP_MJ3h','DDG_V','CP_RMFCA','AP_GOAP_DF']
scoring_functions_rfc = ['CONSRANK_val', 'CP_HLPL', 'CP_MJ3h', 'DDG_V', 'CP_RMFCA', 'AP_GOAP_DF','PRC','TF2']
scoring_functions_nnc = ['CONSRANK_val', 'CP_HLPL', 'CP_MJ3h', 'DDG_V', 'CP_RMFCA', 'AP_GOAP_DF','TF2','RFC']
scoring_functions_dnn = ['CONSRANK_val', 'CP_HLPL', 'CP_MJ3h', 'DDG_V', 'CP_RMFCA', 'AP_GOAP_DF','RFC','PRC']

In [ ]:
feat_rfc = pd.DataFrame([RFC.feature_importances_] , columns= scoring_functions_rfc ) 

In [ ]:
feat_nnc = pd.DataFrame([NCC.coefs_[0].mean(axis=1)] , columns= scoring_functions_nnc ) 

In [ ]:
feat_dnn = pd.DataFrame([DNN.weights[0].numpy().mean(axis=1)] , columns= scoring_functions_dnn  ) 

In [ ]:
print ( pd.concat( [ feat_rfc, feat_nnc,feat_dnn], keys=['RFC','PRC','TF2']  ).to_csv() ) 

,,CONSRANK_val,CP_HLPL,CP_MJ3h,DDG_V,CP_RMFCA,AP_GOAP_DF,PRC,TF2,RFC
RFC,0,0.14005727079365152,0.17810913555857397,0.0734943950476807,0.16473502371583007,0.19207642835705954,0.17126157363234124,0.03663920296381242,0.04362696993105059,
PRC,0,-24.460216806514534,1.1747896372436857,0.08034624261817729,0.47741978768274135,-1.7050382654894267,0.9351304058375213,,-10.868754355128905,2.236822319630757
TF2,0,-8.320362091064453,-0.23365001380443573,0.12053961306810379,-0.0010901808273047209,0.14162257313728333,-0.29330578446388245,-12.443406105041504,,-0.2158188819885254



In [ ]:
std_nnc = pd.DataFrame([NCC.coefs_[0].std(axis=1)] , columns= scoring_functions_nnc ) 
std_dnn = pd.DataFrame([DNN.weights[0].numpy().std(axis=1)] , columns= scoring_functions_dnn  ) 

In [ ]:
print ( pd.concat( [ std_nnc,std_dnn ], keys=['RFC','PRC','TF2']  ).to_csv() ) 

,,CONSRANK_val,CP_HLPL,CP_MJ3h,DDG_V,CP_RMFCA,AP_GOAP_DF,TF2,RFC,PRC
RFC,0,100.40340480131522,6.181770171625957,9.60943994866173,1.075231435534565,8.908526095249519,6.297612816545122,44.57958898586778,5.42285586398655,
PRC,0,24.050437927246094,0.8232786655426025,1.1670814752578735,0.9230668544769287,0.8240094184875488,0.8765534162521362,,0.7926988005638123,16.545391



In [ ]:
pd.concat( [ std_nnc,std_dnn ], keys=['PRC','TF2'] )

,,CONSRANK_val,CP_HLPL,CP_MJ3h,DDG_V,CP_RMFCA,AP_GOAP_DF,TF2,RFC,PRC
PRC,0,100.403405,6.181770,9.609440,1.075231,8.908526,6.297613,44.579589,5.422856,NaN
TF2,0,24.050438,0.823279,1.167081,0.923067,0.824009,0.876553,NaN,0.792699,16.545391


In [ ]:
pd.concat( [ feat_rfc, feat_nnc,feat_dnn], keys=['RFC','PRC','TF2']  ).T

,,CONSRANK_val,CP_HLPL,CP_MJ3h,DDG_V,CP_RMFCA,AP_GOAP_DF,PRC,TF2,RFC
RFC,0,0.140057,0.178109,0.073494,0.164735,0.192076,0.171262,0.036639,0.043627,NaN
PRC,0,-24.460217,1.174790,0.080346,0.477420,-1.705038,0.935130,NaN,-10.868754,2.236822
TF2,0,-8.320362,-0.233650,0.120540,-0.001090,0.141623,-0.293306,-12.443406,NaN,-0.215819
